```{contents}
```
## **State Migration in LangGraph**

**State Migration** in LangGraph is the controlled evolution of a workflow’s **persistent state schema** over time, allowing long-running or saved executions to remain compatible as the system’s data model changes.

It is a core **production reliability mechanism** for maintaining correctness across deployments.

---

### **1. Why State Migration Is Necessary**

LangGraph workflows persist state across:

* executions
* user sessions
* checkpoints
* application versions

When the **State schema changes**, old persisted states may become incompatible with the new code.

Without migration:

* executions crash
* data becomes unreadable
* checkpoints become unusable
* production pipelines break

---

### **2. What “State” Means in LangGraph**

A LangGraph state is:

> A **typed, versioned, persistent data structure** shared across all nodes.

Example:

```python
class State(TypedDict):
    messages: list[str]
    plan: str
    result: str
```

If this schema evolves, migration is required.

---

### **3. Types of State Changes**

| Change Type        | Example                       |
| ------------------ | ----------------------------- |
| Add field          | `confidence: float`           |
| Remove field       | remove `draft`                |
| Rename field       | `answer` → `final_answer`     |
| Change type        | `count: int` → `count: float` |
| Nested restructure | flattening or nesting objects |

Each of these requires explicit migration logic.

---

### **4. Versioning the State Schema**

Production systems attach a **version number** to the state:

```python
class StateV2(TypedDict):
    _version: int
    messages: list[str]
    final_answer: str
    confidence: float
```

All stored states include:

```json
{ "_version": 1, ... }
```

---

### **5. Migration Pipeline Architecture**

```
Load Stored State
       ↓
Check Version
       ↓
Apply Migrations (v1 → v2 → v3 ...)
       ↓
Return Upgraded State
       ↓
Continue Graph Execution
```

---

### **6. Migration Implementation Pattern**

```python
def migrate_v1_to_v2(state):
    state["final_answer"] = state.pop("answer", "")
    state["confidence"] = 0.0
    state["_version"] = 2
    return state
```

```python
def migrate_state(state):
    if state["_version"] == 1:
        state = migrate_v1_to_v2(state)
    return state
```

---

### **7. Integrating Migration into LangGraph Execution**

```python
def entry_node(state):
    state = migrate_state(state)
    return state
```

Set the entry point of the graph to this migration guard.

---

### **8. Checkpoint Compatibility**

When resuming from checkpoints:

* old checkpoints → migrated before execution
* new checkpoints → saved with new version

This preserves:

* replay
* resume
* fault recovery

---

### **9. Safe Migration Rules**

| Rule                | Reason                        |
| ------------------- | ----------------------------- |
| Backward compatible | Do not break running sessions |
| Idempotent          | Safe to apply multiple times  |
| Deterministic       | No randomness                 |
| Explicit            | Never implicit auto-fix       |
| Logged              | Full audit trail              |

---

### **10. Example: Real Evolution Scenario**

**v1**

```python
class State:
    answer: str
```

**v2**

```python
class State:
    final_answer: str
    confidence: float
```

**Migration**

```python
state["final_answer"] = state.pop("answer")
state["confidence"] = 0.95
state["_version"] = 2
```

---

### **11. Why State Migration Enables Production Stability**

Without state migration:

* long-running agents fail
* historical conversations break
* rollbacks impossible
* version upgrades unsafe

With state migration:

> **LangGraph becomes upgrade-safe, checkpoint-safe, and enterprise-ready.**

---

### **12. Mental Model**

LangGraph state behaves like a **database schema** for your agent system.

State migration is the **schema migration engine** for LLM workflows.


### Demonstration

In [7]:
from typing import TypedDict

# v1 schema
class StateV1(TypedDict):
    _version: int
    answer: str

# v2 schema
class StateV2(TypedDict):
    _version: int
    final_answer: str
    confidence: float

def migrate_v1_to_v2(state: dict) -> dict:
    print("Migrating v1 → v2")
    state["final_answer"] = state.pop("final_answer")
    state["confidence"] = 0.95
    state["_version"] = 2
    return state

def migrate_state(state: dict) -> dict:
    if state["_version"] == 1:
        state = migrate_v1_to_v2(state)
    return state


In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    _version: int
    final_answer: str
    confidence: float

def migration_guard(state: dict) -> dict:
    return migrate_state(state)

def process(state: State) -> State:
    state["final_answer"] += " ✅"
    return state


In [9]:
builder = StateGraph(State)

builder.add_node("migrate", migration_guard)
builder.add_node("process", process)

builder.set_entry_point("migrate")
builder.add_edge("migrate", "process")
builder.add_edge("process", END)

graph = builder.compile()


In [10]:
old_state = {
    "_version": 1,
    "final_answer": "Migration successful"
}

result = graph.invoke(old_state)
print(result)


Migrating v1 → v2
{'_version': 2, 'final_answer': 'Migration successful ✅', 'confidence': 0.95}
